In [6]:
from reader import ebook
from main import read_list,read_json
from typing import Optional,Dict,List
import requests
from dotenv import load_dotenv
import json
import os
from pydantic import BaseModel, conint
from dataclasses import dataclass

In [7]:
load_dotenv()
API = os.getenv("HF_API")

headers = {
    "Authorization": f"Bearer {API}",
    "Content-Type": "application/json",
}
url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions"



___

## Get **Summary**, **Characters** AND **Places** 

In [22]:
sum_role='''(NOTE: Only output in JSON. Ensure the JSON format is valid, well-formed, and Ready to parse. nothing before or after the json file)
Input:  
1.Current Chapter Text: The current chapter to be analyzed.
2.Character List: A list of characters with their descriptions till now (This chapter).
3.Places list: list of places and their description till now (This chapter).
4.Previous Chapters' Summary: Context from earlier chapters.

Rules:  
1.Narrative Summary: Summarize and explain the chapter in detail, integrating context and key developments from previous chapters and create a self containing summary and explaination. end with to be continued.
2.Character List: add new characters to the list based on this chapter and Update existing character descriptions. If no characters are mentioned, return the same list as given.
3.Places: Include an updated description of any significant locations mentioned in this chapter, focusing on environment, weather, vibe, and structure.
4.Output Format: Ensure the output is valid and well-structured JSON.

Output:  
Generate a JSON object in this format:
{
  "summary": "Detailed Summary and explination of the current chapter in context of previous chapters. Use previus chapter summary as context",
  "characters": {
      "Character Name": "Updated or new description (age, looks, clothes, hair, body language) based on this chapter."
    },
  "places": {
      "Place Name": "Updated or new description (environment, weather, vibe, structure, etc.) based on this chapter."
  }
}
'''

In [23]:


class SummarySchema(BaseModel):
    summary: str
    characters: Dict[str,str]
    places: Dict[str,str]

def scene_msg(text: str, context: str, characters: dict = {}, places: dict = {}) -> list:
    message = [
        {
            "role": "system",
            "content": sum_role
        },
        {
            "role": "user",
            "content": json.dumps({
                "past_context": context,
                "Current_Chapter": text,
                "character_list": characters,
                "places_list": places
            }),
        },
    ]
    return message

def get_summ(messages:Dict[str,str]) -> Optional[str]:
    data = {
        "messages": messages,
        "temperature": 0.7,
        "stream": False,
        "max_tokens":10000,
        "parameters": {
        "repetition_penalty": 1.3,
        "grammar": {
            "type": "json",
            "value": SummarySchema.model_json_schema()
                }
                    }
    }
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_data = response.json()
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None



In [71]:
# book=ebook("./books/Alchemist/Alchemist.epub")
book=ebook("./books/LP.epub")
title=book.get_metadata()['title']
_,text=zip(*book.get_chapters())

text=text[:3]



In [72]:
output_dict={
    0:
        {
            "summary":"This is the first chapter",
            "characters":{},
            "places": {}
        }
            }

In [73]:
output_dict

{0: {'summary': 'This is the first chapter', 'characters': {}, 'places': {}}}

In [75]:
for idx,i in enumerate(text):
 
    if len(i)<1200:
        output_dict[idx+1]=output_dict[idx]
    else:
        context=output_dict[idx]["summary"]
        characters=output_dict[idx]["characters"]
        places=output_dict[idx]["places"]
        
        mes=scene_msg(i,context,characters,places)
        
        summary_characters=get_summ(mes)
        output_json=read_json(summary_characters)
        output_dict[idx+1]=output_json
    print(f"chapter: {idx} done")
    print(output_dict)

chapter: 0 done
{0: {'summary': 'This is the first chapter', 'characters': {}, 'places': {}}, 1: {'summary': "In this chapter, the author recounts his childhood experiences, particularly his fascination with a drawing of a boa constrictor swallowing an elephant from a book. He reflects on how grown-ups often fail to understand children's perspectives and how this led him to abandon his passion for drawing. Later, he shares an encounter in the Sahara desert where a mysterious little voice requested him to draw a sheep. The chapter provides insight into the author's formative years and his ongoing struggle to communicate his unique vision to the grown-ups around him.", 'characters': {'Grown-ups': "Representative of the adults in the story. They are often portrayed as not understanding children's perspectives and requiring excessive explanations. They discourage the author's passion for drawing.", 'Author (as a child)': 'A character who is fascinated by a drawing of a boa constrictor swal

In [77]:
for num, dic in output_dict.items():
    if num==0:
        continue
    print(f"CHAPTER {num} :\n")
    
    sum,char,places=dic.items()
    print("Characters:")
    if len(char)>0 :
        for name,i in char[1].items():
            print(f"{name}: {i}")
    
    print("\nPlaces:")
    if len(places)>0:
        for name,i in places[1].items():
            print(f"{name}: {i}")
    print(f"\nSummary:\n{sum[1]}")
    print("\n-------------------------------------------------------------------------------------------------------------------")

CHAPTER 1 :

Characters:
Grown-ups: Representative of the adults in the story. They are often portrayed as not understanding children's perspectives and requiring excessive explanations. They discourage the author's passion for drawing.
Author (as a child): A character who is fascinated by a drawing of a boa constrictor swallowing an elephant. He attempts to explain his drawing to the grown-ups but is met with misunderstandings.
Mysterious Little Voice: A new character that appears in the Sahara desert. She requests the author to draw a sheep.

Places:
Child's Room: Setting for the author's initial encounter with the drawing of a boa constrictor swallowing an elephant.
Sahara Desert: A significant location where the author encounters the mysterious little voice. It is described as a harsh, isolated environment.

Summary:
In this chapter, the author recounts his childhood experiences, particularly his fascination with a drawing of a boa constrictor swallowing an elephant from a book. He

___

## Get **Scenes**

In [78]:
class SceneSchema(BaseModel):
    scenes: Dict[str,str]
print(SceneSchema.model_json_schema())

{'properties': {'scenes': {'additionalProperties': {'type': 'string'}, 'title': 'Scenes', 'type': 'object'}}, 'required': ['scenes'], 'title': 'SceneSchema', 'type': 'object'}


In [79]:
def scene_msg(text: str) -> list:
    message = [
        {
            "role": "system",
            "content": ''' IMPORTANT-> ONLY OUTPUT IN JSON.
                You are a text-to-image prompt generator.
                Your task is to analyze the provided input text and identify distinct scenes where there are changes in place or time. For each identified scene, create a detailed and descriptive prompt suitable for generating an image.
                Characters: Refer to characters by their respective names as mentioned in the text.
                Places: Refer to places by their proper names as mentioned in the text.
                Ensure each prompt captures the scene's mood, setting, and key visual elements in detail.
                images are for book
                
                1. **Input:**
                    - `text`: A block of narrative text.```
                2. **Output:**
                    - [prompt,prompt,prompt.....]

                ### Instructions:
                - Identify key changes in location, characters, or significant actions to define separate scenes.
                - Use descriptive language to paint a vivid picture of each scene in the prompt.
                
                   ''',
        },
        
        {
            "role": "user",
            "content": f"""TEXT: {text}""",  # This should be your input text that describes the scenes
        },]
        
    return message

def get_scene(text: str) -> Optional[str]:
    messages = scene_msg(text)
    data = {
        "messages": messages,
        "max_tokens": 10000,  # Specify the maximum length of the response
        "temperature": 0,  # Control the randomness of the response
        "stream": False,
        "repetition_penalty": 1.3,
        "grammar": {
            "type": "json",
            "value": SceneSchema.model_json_schema()
                }
        }
    
    response = requests.post(url, headers=headers, json=data)

    # Check the response status code and process the output
    if response.status_code == 200:
        response_data = response.json()
        # Extract the assistant's message content
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")  # Print error details
        return None

In [80]:
outputs=[]
for i in text:
    inputs=i.replace("\n"," ")
    scenes=get_scene(inputs)
    scene_json_output=read_json(scenes)
    outputs.append(scene_json_output)

___


## Get **Style**

In [81]:
def basic_llm_req(text: str) -> Optional[str]:
    messages = [
        {
            "role":"system",
            "content":"DO As Asked in The Input"
        },
        {
            "role":"user",
            "content":text
        }
    ]
    
    data = {
        "messages": messages,
        "max_tokens": 10000,  # Specify the maximum length of the response
        "temperature": 0,  # Control the randomness of the response
        "stream": False,
    }
    response = requests.post(url, headers=headers, json=data)

    # Check the response status code and process the output
    if response.status_code == 200:
        response_data = response.json()
        # Extract the assistant's message content
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")  # Print error details
        return None
 

In [82]:
combined_summary=""
for key,val in output_dict.items():
    val_content=val["summary"].replace("\n"," ")
    if key==0:
        continue
    output_string= f'''{combined_summary}... Chapter{key}: {val_content}'''

In [83]:
style_prompt='''Prompt:

"Analyze the following story and provide a list of image style tags that would best suit its themes, settings, and overall mood. 
The response should include the style, period, type of art, color palette etc. 1 tags per entry.dont explain it just give tags.

Response Format:

Style: Realism, Impressionism, Surrealism,etc
Type:Landscape, Portrait, Abstract,etc.
Color Palette:Warm tones, Cool colors, Monochromatic,etc
Mood:Serene, Dramatic, Melancholic,etc.

Story
'''

style=basic_llm_req(f'''{style_prompt}:{combined_summary} ''')

In [85]:
style=" Style:Impressionism\n2. Type: Landscape\n3. Color Palette: Cool colors (Blues, Greens, Purples)\n4. Mood: Melancholic, Nostalgic"

___

## Get **Image**

In [86]:
from huggingface_hub import InferenceClient
import time

client = InferenceClient(
    "stabilityai/stable-diffusion-3.5-large-turbo",
    token=API,
)

In [87]:
def get_images(key, text,characters,places,style):
    image = client.text_to_image(
        f"Generate a illustration for a book given the prompt,use provided character place descriptions if needed. Prompt: {text}, characters: {characters}, places:{places}// style:{style} ",
        height=528,
        width=720,
    )
    image.save(f"./output/{title}_{key}.png")

In [89]:
outputs

[[{'scene': "A young boy's bedroom in France, winter",
   'description': "A cozy, dimly lit room filled with the soft glow of a fireplace. The room is sparsely furnished with a single bed, a small wooden desk, and a few books scattered around. The boy, bundled up in blankets, is huddled under the covers, looking cold and hungry. The mood is somber and melancholic, reflecting the boy's discomfort and longing."},
  {'scene': 'A library, undisclosed location',
   'description': 'A grand, ornate library filled with towering bookshelves, intricate carvings, and a large, open fireplace. A grown-up, possibly the same boy from the previous scene, is engrossed in a book about the virgin forest. The mood is contemplative and curious, as the grown-up is lost in thought, inspired by the illustrations of a boa constrictor swallowing a wild beast. The key visual elements are the detailed book illustrations and the warm, inviting fireplace.'},
  {'scene': "A young boy's bedroom, undisclosed location"

In [ ]:

for idx,i in enumerate(outputs):
    chars=output_dict[idx+1]["characters"]
    places=output_dict[idx+1]["places"]
    for jdx,j in enumerate(i):
        ##try except for internal server error
        try:
            get_images(key=f"chapter{idx}_scene{jdx+1}",text=j,characters=chars,places=places,style=style)
            print(f"chapter{idx}_scene{jdx+1} saved")
        except Exception as e:
            print(f"Error while calling the API :{e} \n waiting for 10 seconds.. ")
            time.sleep(10)
            get_images(key=f"chapter{idx}_scene{jdx+1}",text=j,characters=chars,places=places,style=style)
            print(f"chapter{idx}_scene{jdx+1} saved")
        time.sleep(18)

chapter0_scene1 saved
chapter0_scene2 saved
chapter0_scene3 saved
chapter0_scene4 saved
chapter0_scene5 saved


HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-3.5-large-turbo (Request ID: OWwzYmu7igYIo99xVVboP)